Guilherme Pereira Albino da Silva      RA: 182786 <br>
Willian Félix Souza e Silva            RA: 218415

# Aspectos Básicos dos Fluxos de Carga

## Import das bibliotecas

In [ ]:
import numpy as np
import pandas as pd

## Lendo os arquivos e convertendo para Pandas DataFrame

In [ ]:
BusData = np.loadtxt('ieee5bus.txt',skiprows=2)
BranchData = np.loadtxt('ieee5branch.txt',skiprows=2)

In [ ]:
df_bus = pd.DataFrame(BusData,columns=['N', 'Tipo', 'Tensao', 'Theta', 'PC', 'QC', 'PG', 'QG', 'Bsh'])
df_branch = pd.DataFrame(BranchData,columns=['DE', 'PARA', 'r', 'x', 'bsh', 'Tap', 'Phi'])

In [ ]:
df_bus

,N,Tipo,Tensao,Theta,PC,QC,PG,QG,Bsh
0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,-0.0,0.0
1,2.0,0.0,1.0,0.0,40.0,20.0,0.0,0.0,0.0
2,3.0,0.0,1.0,0.0,25.0,15.0,0.0,0.0,0.0
3,4.0,0.0,1.0,0.0,40.0,20.0,0.0,0.0,0.0
4,5.0,0.0,1.0,0.0,50.0,20.0,0.0,0.0,0.0


In [ ]:
df_branch

,DE,PARA,r,x,bsh,Tap,Phi
0,1.0,2.0,0.05,0.11,0.02,1.0,0.0
1,1.0,3.0,0.05,0.11,0.02,1.0,0.0
2,1.0,5.0,0.03,0.08,0.02,1.0,0.0
3,2.0,3.0,0.04,0.09,0.02,1.0,0.0
4,2.0,5.0,0.04,0.09,0.02,1.0,0.0
5,3.0,4.0,0.06,0.13,0.03,1.0,0.0
6,4.0,5.0,0.04,0.09,0.02,1.0,0.0


## Criando matrizes de impedância e admitância

In [ ]:
k_bus, m_bus = df_bus.shape
k_bra, m_bra = df_branch.shape

print(f'Número de k: {k_bus}, Número de conexões: {k_bra}')

Número de k: 5, Número de conexões: 7


In [ ]:
Y = np.zeros((k_bus,k_bus), dtype=np.complex128)
Z = np.zeros((k_bus,k_bus), dtype=np.complex128)
Y_read = np.zeros((k_bus,k_bus), dtype=np.complex128)
yk = np.zeros((k_bus,k_bus), dtype=np.complex128)

a = np.zeros((k_bus,k_bus), dtype=float)
b = np.zeros((k_bus,k_bus), dtype=float)
E = np.zeros((k_bus), dtype=float)
theta = np.zeros((k_bus), dtype=float)

### IMPEDÂNCIA

In [ ]:
for i in range(k_bra):
    
    k = df_branch.loc[i,['DE']].to_numpy(dtype=int)[0]-1
    m = df_branch.loc[i,['PARA']].to_numpy(dtype=int)[0]-1
    
    r = df_branch.loc[i,['r']].to_numpy(dtype=float)[0]
    x = df_branch.loc[i,['x']].to_numpy(dtype=float)[0]
    
    z = complex(r, x)
    
    Z[k, m] = z
    Z[m, k] = z
    
    a[k, m] = df_branch.loc[i,['Tap']].to_numpy(dtype=float)[0]
    a[m, k] = 1 / (df_branch.loc[i,['Tap']].to_numpy(dtype=float)[0])
    
    b[k, m] = df_branch.loc[i,['bsh']].to_numpy(dtype=float)[0]/2
    b[m, k] = df_branch.loc[i,['bsh']].to_numpy(dtype=float)[0]/2
    
for i in range(k_bus):
    b[i, i] = df_bus.loc[i,['Bsh']].to_numpy(dtype=float)[0]
    E[i] = df_bus.loc[i,['Tensao']].to_numpy(dtype=float)[0]
    theta[i] = df_bus.loc[i,['Theta']].to_numpy(dtype=float)[0]

### ADMITÂNCIA

In [ ]:
for i in range(k_bus):
    for j in range(k_bus):
        
        if Z[i,j] != 0 + 0j:
            yk[i,j] = 1 / Z[i,j]
            Y[i,j] = -a[i,j]*(1 / Z[i,j])
            Y_read[i,j] = np.round(-a[i,j]*(1 / Z[i,j]), 2)
            
        else:
            yk[i,j] = 0 + 0j
            Y[i,j] = 0 + 0j
            Y_read[i,j] = 0 + 0j
            
            
for i in range(k_bus):
    
    soma = 0
    
    for j in range(k_bus):
        
        if j != i:
        
            soma += complex(0,b[i,j]) + (a[i,j]**2) * yk[i,j]
            
    soma = soma + complex(0,b[i,i])

    Y[i,i] = soma
    Y_read[i,i] = np.round(soma, 6)

In [ ]:
matriz_size = [str(i+1) for i in range(k_bus)]
print(matriz_size)

df_Y_read = pd.DataFrame(Y_read, columns = matriz_size, index = matriz_size)
df_Z_read = pd.DataFrame(Z, columns = matriz_size, index = matriz_size)

['1', '2', '3', '4', '5']


In [ ]:
df_Y_read

,1,2,3,4,5
1,10.958904-25.997397j,-3.420000+7.530000j,-3.420000+7.530000j,0.000000+0.000000j,-4.110000+10.960000j
2,-3.420000+7.530000j,11.672080-26.060948j,-4.120000+9.280000j,0.000000+0.000000j,-4.120000+9.280000j
3,-3.420000+7.530000j,-4.120000+9.280000j,10.475198-23.119061j,-2.930000+6.340000j,0.000000+0.000000j
4,0.000000+0.000000j,0.000000+0.000000j,-2.930000+6.340000j,7.050541-15.594814j,-4.120000+9.280000j
5,-4.110000+10.960000j,-4.120000+9.280000j,0.000000+0.000000j,-4.120000+9.280000j,12.357012-29.485605j


## Corrente

In [ ]:
I = np.zeros((k_bus), dtype=np.complex128)
P = np.zeros((k_bus), dtype=np.complex128)

In [ ]:
I = np.zeros((k_bus), dtype=np.complex128)
for i in range(k_bus):
    for j in range(k_bus):
        
        if i == j:
            I[i] += Y[i,j]*E[i]
        else:
            I[i] += Y[i,j]*E[j]

## Cálculo das potências

### Pela corrente

In [ ]:
S = np.zeros((k_bus), dtype=np.complex128)
for i in range(k_bus):
    S[i] = np.conj(I[i]) * E[i]

for i in range(len(S)):
    print(f'S{i} = {S[i]}')

S0 = (8.881784197001252e-16-0.02999999999999936j)
S1 = -0.029999999999997584j
S2 = -0.034999999999998366j
S3 = -0.02499999999999858j
S4 = -0.029999999999997584j


### Pela equação

In [ ]:
T = np.zeros((k_bus), dtype=np.complex128)
Q = np.zeros((k_bus), dtype=float)
P = np.zeros((k_bus), dtype=float)
for i in range(k_bus):
  
    for j in range(k_bus):
        T[i] += np.real(Y[i,j])
 

        P[i] += E[j]*(np.real(Y[i,j])*np.cos(theta[i] - theta[j]) + np.imag(Y[i,j])*np.sin(theta[i] - theta[j]))
        
        Q[i] += E[j]*(np.real(Y[i,j])*np.sin(theta[i] - theta[j]) - np.imag(Y[i,j])*np.cos(theta[i] - theta[j]))

        
    P[i] = E[i]*P[i]
    Q[i] = E[i]*Q[i]


for i in range(len(P)):
    print(f'P{i+1} = {P[i]:.5}\tQ{i+1} = {Q[i]:.5}')

P1 = 8.8818e-16	Q1 = -0.03
P2 = 0.0	Q2 = -0.03
P3 = 0.0	Q3 = -0.035
P4 = 0.0	Q4 = -0.025
P5 = 0.0	Q5 = -0.03


In [ ]:
print(theta)

[0. 0. 0. 0. 0.]


In [ ]:
Q = np.zeros(k_bus, dtype=float)
P = np.zeros(k_bus, dtype=float)

for i in range(5):
  for j in range(5):
    gkm = np.real(Y[i,j])
    bkm = np.imag(Y[i,j])
    thetakm = theta[i] - theta[j]
    print(gkm,bkm)
    
    P[i] = P[i] + E[j]*(gkm*np.cos(thetakm) + bkm*np.sin(thetakm))
    Q[i] = Q[i] + E[j]*(gkm*np.sin(thetakm) - bkm*np.cos(thetakm))

  P[i] = E[i]*P[i]
  Q[i] = E[i]*Q[i]

for i in range(len(P)):
    print(f'P{i+1} = {P[i]:.5}\tQ{i+1} = {Q[i]:.5}')

10.958904109589042 -25.99739726027397
-3.4246575342465757 7.534246575342466
-3.4246575342465757 7.534246575342466
0.0 0.0
-4.10958904109589 10.95890410958904
-3.4246575342465757 7.534246575342466
11.672080214658948 -26.060947606270304
-4.123711340206186 9.278350515463918
0.0 0.0
-4.123711340206186 9.278350515463918
-3.4246575342465757 7.534246575342466
-4.123711340206186 9.278350515463918
10.475198142745445 -23.119060505440533
-2.926829268292683 6.341463414634147
0.0 0.0
0.0 0.0
0.0 0.0
-2.926829268292683 6.341463414634147
7.050540608498869 -15.594813930098066
-4.123711340206186 9.278350515463918
-4.10958904109589 10.95890410958904
-4.123711340206186 9.278350515463918
0.0 0.0
-4.123711340206186 9.278350515463918
12.357011721508261 -29.485605140516878
P1 = 8.8818e-16	Q1 = -0.03
P2 = 0.0	Q2 = -0.03
P3 = 0.0	Q3 = -0.035
P4 = 0.0	Q4 = -0.025
P5 = 0.0	Q5 = -0.03
